In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
%config InlineBackend.figure_format = "retina"

from scipy.signal import stft, istft, get_window
from scipy.fftpack import fft, fftshift, fftfreq
from IPython.display import Audio
from tqdm import tnrange, tqdm_notebook, tqdm
from dlbeamformer_utilities import *
from dlbeamformers import *
from utilities import *
from IPython.display import Audio

random_seed = 0
# Make pretty figures
palette, cmap = config_figures()

VISUALIZE_BEAMPATTERNS = True
plt.style.use("seaborn-poster")
main_color = (59/255, 157/255, 180/255)

ModuleNotFoundError: No module named 'utilities'

In [ ]:
datapath = "CMU_ARCTIC/cmu_us_bdl_arctic/wav"
train_data, test_data = load_data(datapath)

sampling_frequency, stft_params, sound_speed = parse_parameters()

signal_max_frequency = sampling_frequency / 2

# Array geometry
pos_x = np.array([-35.0, -35.0, 0.0, 35.0, 35.0, 0.0, 0.0]) * 1e-3
pos_y = np.array([20.0, -20.0, -40.0, -20.0, 20.0, 40.0, 0.0]) * 1e-3
n_mics = len(pos_x)
pos_z = np.zeros(n_mics)
array_geometry = np.row_stack((pos_x, pos_y, pos_z))

# Fix elevation angle
elevation = -90 # [degree]

# Source/Target/Look angles
elevation_s = np.array([elevation]) # [degree]
azimuth_s = np.array([180])
source_steering_vectors = compute_steering_vectors(array_geometry, 
        sampling_frequency=sampling_frequency, n_fft_bins=stft_params["n_fft_bins"], 
        elevation_grid=elevation_s, 
        azimuth_grid=azimuth_s)

# Scanning angles
scanning_elevation_grid = np.array([elevation]) # [degree]
scanning_azimuth_grid = np.arange(0, 360, 0.1) # [degree]
scanning_steering_vectors = compute_steering_vectors(array_geometry, 
        sampling_frequency=sampling_frequency, n_fft_bins=stft_params["n_fft_bins"], 
        elevation_grid=scanning_elevation_grid, 
        azimuth_grid=scanning_azimuth_grid)

In [ ]:
ds_tf_beamformers = 1./n_mics * source_steering_vectors

# Delay-sum beam pattern
ds_tf_beampattern = compute_tf_beampattern(ds_tf_beamformers[:, 0, 0, :], scanning_steering_vectors)
ds_tf_beampattern_db = to_db(ds_tf_beampattern)

if VISUALIZE_BEAMPATTERNS:
    frequency_bins = [7, 31, 63, 127]
    visualize_beampattern_1d(ds_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, title="Delay-sum TF beam patterns")

    visualize_beampattern_1d_average(np.abs(ds_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, title="Delay-sum TF average beam pattern")

    visualize_beampattern_2d(np.abs(ds_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);

In [ ]:
np.random.seed(random_seed)
n_interferences = 1
azimuth_step = 5
n_samples_each_config = len(train_data)
training_azimuths = list(np.arange(0, 360, azimuth_step))
training_elevations = np.array([-90])
training_angles = {
    "elevation": training_elevations,
    "azimuth": training_azimuths
}
noisy = False
training_tf_filename = \
    "circular_CMU_ARCTIC_tf_training_covariance_data_azimuth_step_{}_config_trainning_samples_{}_n_interferences_{}.pkl".format(
        azimuth_step, n_samples_each_config, n_interferences
    )
if noisy == True:
    training_tf_filename = "noisy_" + training_tf_filename
training_tf_path = "/data/dung/dlbeamformer/tf_training_data"
training_tf_filepath = os.path.join(training_tf_path, training_tf_filename)

training_interference_covariance = prepare_multichannel_covariance_data(
                    array_geometry, train_data, n_interferences, training_angles,
                    azimuth_step, n_samples_each_config, sampling_frequency,
                    stft_params, training_tf_filepath, random_seed=0, SAVE=False)

source_angles = [
    {
        "elevation": elevation_s,
        "azimuth": azimuth_s
    }
]
angle_grid = {
    "elevation": scanning_elevation_grid,
    "azimuth": scanning_azimuth_grid
}
dlbeamformer = DictionaryLearningBeamformer(array_geometry, sampling_frequency,
        source_angles, stft_params, angle_grid, diagonal_loading_param=5, bf_type="NC")
desired_null_width = 5 # [degree]
null_constraint_threshold = 0.001
n_atoms_each_config = 1
batch_size = 100
dlbeamformer.fit(training_interference_covariance, desired_null_width, 
                null_constraint_threshold, eigenvalue_percentage_threshold=0.99,
                batch_size=batch_size, n_atoms_each_config=n_atoms_each_config)
dlbeamformer_filename = \
    "circular_CMU_ARCTIC_dlbeamformer_azimuth_step_{}_config_atoms_{}_n_interferences_{}_batch_size_{}.pkl".format(
        azimuth_step, n_atoms_each_config, n_interferences, batch_size)
trained_models_path = "/data/dung/dlbeamformer/trained_models"
dlbeamformer_filepath = os.path.join(trained_models_path, dlbeamformer_filename)

In [ ]:
# import importlib
# import dlbeamformers
# importlib.reload(dlbeamformers)
# from dlbeamformers import *
# batch_size = 100
# dl = DLBeamformer(array_geometry, sampling_frequency,
#         source_angles, stft_params, angle_grid, diagonal_loading_param=5,
#         n_dict_atoms=len(training_azimuths), n_nonzero_coefficients=10, 
#         n_train_max_iterations=1, train_error_tolerance=1e-5)
# dl.fit(training_interference_covariance, desired_null_width, 
#                 null_constraint_threshold, eigenvalue_percentage_threshold=0.99,
#                 batch_size=batch_size, n_train_batches_each_config=3)

In [ ]:
# plt.plot(dl.training_loss)

In [ ]:
# print(dl.weights_[0].shape)
# print(dl.weights_[1].shape)
# dw = dl.weights_[2]
# w = np.zeros((256, 7, 1), dtype=np.complex64)
# # w2 = np.zeros((256, 7, 1), dtype=np.complex64)
# for i in range(256):
#     w[i] = dl.weights_[0][0][i].dot(dl.weights_[1][0, i, :, 15]).reshape(7, 1)
# #     w2[i] = dl.weights_[0][0][i].dot(dl.weights_[1][0, i, :, 1]).reshape(7, 1)

In [ ]:
np.random.seed(random_seed)
n_interferences = 1
# input_sinrs_db = np.arange(-15, 16, 2)
input_sinrs_db = np.array([0])
input_inr_db = 0
input_inr = from_db(input_inr_db)
beamformer_list = ["DL-MPDR", "DL-MVDR", "DS", "MVDR", "MPDR"]
# beamformer_list = ["mvdr"]
beamformers = {}
for beamformer_name in beamformer_list:
    beamformers[beamformer_name] = {
        "weights": [None]*len(input_sinrs_db),
        "sinr_db": [None]*len(input_sinrs_db),
        "average_sinr_db": [],
        "out": [None]*len(input_sinrs_db),
        "ir_db": [None] * len(input_sinrs_db),
        "average_ir_db": [],
        "nr_db": [None] * len(input_sinrs_db),
        "average_nr_db": [],
        "sr_db": [None] * len(input_sinrs_db),
        "average_sr_db": []
    }
        
n_MC_iters = 1
for i_input_sinr in tqdm(range(len(input_sinrs_db)), desc="Input SINR"):
    input_sinr = from_db(input_sinrs_db[i_input_sinr])
    for beamformer_name in beamformer_list:
        beamformers[beamformer_name]["weights"][i_input_sinr] = []
        beamformers[beamformer_name]["sinr_db"][i_input_sinr] = []
        beamformers[beamformer_name]["out"][i_input_sinr] = []
        beamformers[beamformer_name]["ir_db"][i_input_sinr] = []
        beamformers[beamformer_name]["nr_db"][i_input_sinr] = []
        beamformers[beamformer_name]["sr_db"][i_input_sinr] = []
    for i_MC_iter in tqdm(range(n_MC_iters), desc="Monte Carlo iterations"):
    
        source = {
            "signal": test_data[np.random.choice(len(test_data))],
            "elevation": elevation_s,
            "azimuth": azimuth_s
        }
        
        interferences = []
        interference_signals = []
        for i_interference in range(n_interferences):
            random_azimuth_1 = np.random.uniform(scanning_azimuth_grid[0], azimuth_s - 30)[0]
            random_azimuth_2 = np.random.uniform(azimuth_s + 30, scanning_azimuth_grid[-1])[0]
            random_azimuth = np.random.choice(np.array([random_azimuth_1, random_azimuth_2]))
            interference_signal = test_data[np.random.choice(len(test_data))]
            interference = {
                "signal": interference_signal,
                "elevation": np.array([elevation]),
#                 "azimuth": np.array([np.random.uniform(
#                     scanning_azimuth_grid[0], scanning_azimuth_grid[-1])])
                "azimuth": np.array([32])
#                 "azimuth": np.array([random_azimuth])
            }
            interferences.append(interference)
        
        received_stft_multichannel, source_stft_multichannel, \
        interference_stft_multichannel, noise_stft_multichannel \
            = simulate_multichannel_tf_mixtures(array_geometry, source,
                interferences, sampling_frequency, stft_params, input_inr, input_sinr)
        
        for beamformer_name in beamformer_list:        
            if beamformer_name.lower() == "ds":
                tf_frames_multichannel = None
            elif beamformer_name.lower() in ["mvdr", "dl-mvdr"]:
                tf_frames_multichannel = interference_stft_multichannel
            elif beamformer_name.lower() in ["mpdr", "dl-mpdr"]:
                tf_frames_multichannel = received_stft_multichannel

            if beamformer_name.lower() in ["dl-mvdr", "dl-mpdr"]:
                source_angle_index = 0
                tf_beamformer = dlbeamformer.choose_weights(source_angle_index,
                                    tf_frames_multichannel)
            else:
                tf_beamformer = compute_tf_beamformers(source_steering_vectors[:, 0, 0, :], 
                        beamformer_name=beamformer_name,
                        tf_frames_multichannel=tf_frames_multichannel,
                        diagonal_loading_param=5)

            # Compute beamformer output and SINR
            tf_out, out, _ = compute_tf_beamformer_output(tf_beamformer, 
                                received_stft_multichannel, sampling_frequency, 
                                stft_params)
            
            sinr_db, sinr  = compute_sinr(source_stft_multichannel, 
                interference_stft_multichannel+noise_stft_multichannel, tf_beamformer)

            # Compute interference output and interference reduction
            interference_tf_out, interference_out, _ = compute_tf_beamformer_output(tf_beamformer, 
                                interference_stft_multichannel, sampling_frequency, 
                                stft_params)
            
            interference_reduction = compute_power(interference_stft_multichannel[:, 0, :]) / compute_power(interference_tf_out)
            interference_reduction_db = to_db(interference_reduction)
            
            # Compute noise output and noise reduction
            noise_tf_out, noise_out, _ = compute_tf_beamformer_output(tf_beamformer, 
                                noise_stft_multichannel, sampling_frequency, 
                                stft_params)
            
            noise_reduction = compute_power(noise_stft_multichannel[:, 0, :]) / compute_power(noise_tf_out)
            noise_reduction_db = to_db(noise_reduction)
            
            # Compute source output and source distortion/reduction
            source_tf_out, source_out, _ = compute_tf_beamformer_output(tf_beamformer, 
                                source_stft_multichannel, sampling_frequency, 
                                stft_params)
            
            source_reduction = compute_power(source_stft_multichannel[:, 0, :]) / compute_power(source_tf_out)
            source_reduction_db = to_db(source_reduction)
            
            beamformers[beamformer_name]["weights"][i_input_sinr].append(tf_beamformer)
            beamformers[beamformer_name]["sinr_db"][i_input_sinr].append(sinr_db[0][0])
            beamformers[beamformer_name]["out"][i_input_sinr].append(out)
            beamformers[beamformer_name]["ir_db"][i_input_sinr].append(interference_reduction_db)
            beamformers[beamformer_name]["nr_db"][i_input_sinr].append(noise_reduction_db)
            beamformers[beamformer_name]["sr_db"][i_input_sinr].append(source_reduction_db)

    for beamformer_name in beamformers.keys():
        beamformers[beamformer_name]["average_sinr_db"].append(
            to_db(np.mean(from_db(np.asarray(beamformers[beamformer_name]["sinr_db"][i_input_sinr])))))
        beamformers[beamformer_name]["average_ir_db"].append(
            to_db(np.mean(from_db(np.asarray(beamformers[beamformer_name]["ir_db"][i_input_sinr])))))
        beamformers[beamformer_name]["average_nr_db"].append(
            to_db(np.mean(from_db(np.asarray(beamformers[beamformer_name]["nr_db"][i_input_sinr])))))
        beamformers[beamformer_name]["average_sr_db"].append(
            to_db(np.mean(from_db(np.asarray(beamformers[beamformer_name]["sr_db"][i_input_sinr])))))

In [ ]:
ss = beamformers["MVDR"]["out"][0][0]
data=np.int16(ss/np.max(np.abs(ss)) * 32767)
display(Audio(data, rate=sampling_frequency, autoplay=True))

# write_audio_file(ss, sampling_frequency, "test.wav")  

In [ ]:
# figsize = (9, 6)
# fig = plt.figure(figsize=figsize); ax = fig.add_subplot(111);
# for beamformer_name in beamformer_list:
#     if beamformer_name == "DL-MPDR":
#         ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_sinr_db"], 
#                 color=main_color, label=beamformer_name)
#     else:
#         ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_sinr_db"],
#                 label=beamformer_name)
# ax.set_xlim(input_sinrs_db[0], input_sinrs_db[-1]);
# ax.set_xlabel("Input SINR"); ax.set_ylabel("Output SINR [dB]");
# plt.legend();

# fig = plt.figure(figsize=figsize); ax = fig.add_subplot(111);
# for beamformer_name in beamformer_list:
#     if beamformer_name == "DL-MPDR":
#         ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_sr_db"], 
#                 color=main_color, label=beamformer_name)
#     else:
#         ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_sr_db"], label=beamformer_name)
# ax.set_xlim(input_sinrs_db[0], input_sinrs_db[-1]);
# ax.set_ylim(-20, 20)
# ax.set_xlabel("Input SINR"); ax.set_ylabel("Source distortion [dB]");
# plt.legend();

# fig = plt.figure(figsize=figsize); ax = fig.add_subplot(111);
# for beamformer_name in beamformer_list:
#     if beamformer_name == "DL-MPDR":
#         ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_ir_db"], 
#                 color=main_color, label=beamformer_name)
#     else:
#         ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_ir_db"], label=beamformer_name)
# ax.set_xlim(input_sinrs_db[0], input_sinrs_db[-1]);
# ax.set_xlabel("Input SINR"); ax.set_ylabel("Interference reduction [dB]");
# plt.legend();

# fig = plt.figure(figsize=figsize); ax = fig.add_subplot(111);
# for beamformer_name in beamformer_list:
#     if beamformer_name == "DL-MPDR":
#         ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_nr_db"], 
#                 color=main_color, label=beamformer_name)
#     else:
#         ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_nr_db"], label=beamformer_name)
# ax.set_xlim(input_sinrs_db[0], input_sinrs_db[-1]);
# ax.set_xlabel("Input SINR"); ax.set_ylabel("Noise reduction [dB]");
# plt.legend();
# plt.tight_layout()
# fig_name = "figures/benchmark_dl_inputINR_{}dB.jpg".format(input_inr_db)
# # plt.savefig(fig_name, dpi=600)

In [ ]:
fig_name = "benchmark_dl_inputINR_{}db.jpg".format(input_inr_db)

In [ ]:
play_tf_frames(received_stft_multichannel[:, 0, :], sampling_frequency, stft_params)
# Audio(interference["signal"], rate=sampling_frequency, autoplay=True)

In [ ]:
stft_window = stft_params["window"]
n_samples_per_frame = stft_params["n_samples_per_frame"]
hop_size = stft_params["hop_size"]
t, ss = istft(source_stft_multichannel[:, 0, :], fs=sampling_frequency, 
    window=stft_window, nperseg=n_samples_per_frame, 
    noverlap=n_samples_per_frame-hop_size,
    nfft=n_samples_per_frame, boundary=True)
Audio(ss, rate=sampling_frequency, autoplay=True)
# plt.plot(ss)

In [ ]:
# frequency_bins = [7, 31, 63, 127]
# dl_beampattern = compute_tf_beampattern(beamformers["dlbeamformer"]["weights"], scanning_steering_vectors)
# dl_beampattern_db = to_db(dl_beampattern)

# visualize_beampattern_1d(dl_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
#     signal_max_frequency, source_azimuths= azimuth_s, title="DL TF beam patterns")

# visualize_beampattern_1d_average(np.abs(dl_beampattern[:, 0, :]), scanning_azimuth_grid, 
#     frequency_range=(0, 127), source_azimuths=azimuth_s, title="DL TF average beam pattern")

# visualize_beampattern_2d(np.abs(dl_beampattern[:, 0, :]), 
#     scanning_azimuth_grid, signal_max_frequency);

# def visualize_beampatterns(beamformer, scanning_steering_vector, scanning_azimuth_grid, 
#         source_azimuths, signal_max_frequency, frequency_bins, frequency_range, title=None):
#     beampattern = compute_tf_beampattern(beamformer, scanning_steering_vectors)
#     beampattern_db = to_db(beampattern)
    
#     visualize_beampattern_1d(dl_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
#         signal_max_frequency, source_azimuths= azimuth_s, title="DL TF beam patterns")

#     visualize_beampattern_1d_average(np.abs(dl_beampattern[:, 0, :]), scanning_azimuth_grid, 
#         frequency_range=(0, 127), source_azimuths=azimuth_s, title="DL TF average beam pattern")

#     visualize_beampattern_2d(np.abs(dl_beampattern[:, 0, :]), 
#         scanning_azimuth_grid, signal_max_frequency);
    


In [ ]:
# beamformer_list = ["delaysum", "mvdr", "mpdr", "dlbeamformer"]
# beamformer_name = beamformer_list[2]
# # len(beamformers[beamformer_name]["out"])
# Audio(beamformers[beamformer_name]["out"][2][0], rate=sampling_frequency, autoplay=True)
# # play_tf_frames(interference_stft_multichannel[:, 0, :]+noise_stft_multichannel[:, 0, :], sampling_frequency, stft_params)
# # play_tf_frames(received_stft_multichannel[:, 0, :], sampling_frequency, stft_params)

In [ ]:
# beamformers["mvdr"]["weights"][3][0].shape

In [ ]:
beamformers["MVDR"]["weights"][0][0]

In [ ]:
# MVDR beam pattern
mvdr_tf_beampattern = compute_tf_beampattern(beamformers["MVDR"]["weights"][0][0], scanning_steering_vectors)
mvdr_tf_beampattern_db = to_db(mvdr_tf_beampattern)

if VISUALIZE_BEAMPATTERNS:
    frequency_bins = [7, 31, 63, 127]
    visualize_beampattern_1d(mvdr_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, title="MVDR TF beam patterns")

    visualize_beampattern_1d_average(np.abs(mvdr_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, title="MVDR TF average beam pattern")

    visualize_beampattern_2d(np.abs(mvdr_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);

In [ ]:
print(w.shape)
dl_tf_beampattern = compute_tf_beampattern(w, scanning_steering_vectors)
dl_tf_beampattern_db = to_db(dl_tf_beampattern)

if VISUALIZE_BEAMPATTERNS:
    frequency_bins = [7, 31, 63, 127]
    visualize_beampattern_1d(dl_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, title="MVDR TF beam patterns")

    visualize_beampattern_1d_average(np.abs(dl_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, title="MVDR TF average beam pattern")

    visualize_beampattern_2d(np.abs(dl_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);

In [ ]:
print(dw.shape)
dl_tf_beampattern = compute_tf_beampattern(dw[0, :, :, 15], scanning_steering_vectors)
dl_tf_beampattern_db = to_db(dl_tf_beampattern)

if VISUALIZE_BEAMPATTERNS:
    frequency_bins = [7, 31, 63, 127]
    visualize_beampattern_1d(dl_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, title="MVDR TF beam patterns")

    visualize_beampattern_1d_average(np.abs(dl_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, title="MVDR TF average beam pattern")

    visualize_beampattern_2d(np.abs(dl_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);

In [ ]:
dlbeamformer.weights_.shape
w2 = dlbeamformer.weights_[0, :, :, 15]
dl_tf_beampattern = compute_tf_beampattern(dw, scanning_steering_vectors)
dl_tf_beampattern_db = to_db(dl_tf_beampattern)

if VISUALIZE_BEAMPATTERNS:
    frequency_bins = [7, 31, 63, 127]
    visualize_beampattern_1d(dl_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, title="MVDR TF beam patterns")

    visualize_beampattern_1d_average(np.abs(dl_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, title="MVDR TF average beam pattern")

    visualize_beampattern_2d(np.abs(dl_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);

In [ ]:
# import matplotlib as mpl
# mpl.rcParams['grid.linewidth'] = 0.3
# mpl.rcParams['grid.alpha'] = 0.3
# folder = "figures"
# filename = "sinr_input_inr_{}dB".format(input_inr_db)
# filepath = os.path.join(folder, filename + ".jpg")
# figsize = (9, 6)
# fig = plt.figure(figsize=figsize); ax = fig.add_subplot(111);
# for i_beamformer, beamformer_name in enumerate(beamformer_list):
#     ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_sinr_db"],
#                 label=beamformer_name, color=palette[i_beamformer])
#     y_pos = beamformers[beamformer_name]["average_sinr_db"][-1] - 0.5 
#     if beamformer_name == "DL-MVDR":
#         y_pos -= 1
#     if beamformer_name == "MPDR":
#         y_pos -= 1
#     ax.text(input_sinrs_db[-1]+0.5, y_pos, beamformer_name, fontsize=12, color=palette[i_beamformer])
# ax.fill_between(input_sinrs_db, beamformers["DL-MPDR"]["average_sinr_db"], 
#                 beamformers["MPDR"]["average_sinr_db"], color=palette[0], alpha=0.2)
# ax.set_xlim(input_sinrs_db[0], input_sinrs_db[-1]);
# ax.set_xlabel("Input SINR"); ax.set_ylabel("Output SINR [dB]");
# ax.spines["top"].set_alpha(0.3)
# ax.spines["bottom"].set_alpha(0.3)
# ax.spines["right"].set_alpha(0.3)
# ax.spines["left"].set_alpha(.3)
# fig.savefig(filepath, dpi=600)

# filename = "sr_input_inr_{}dB".format(input_inr_db)
# filepath = os.path.join(folder, filename + ".jpg")
# fig = plt.figure(figsize=figsize); ax = fig.add_subplot(111);
# for i_beamformer, beamformer_name in enumerate(beamformer_list):
#     ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_sr_db"],
#                 label=beamformer_name, color=palette[i_beamformer])
#     y_pos = beamformers[beamformer_name]["average_sr_db"][-1] - 0.5 
# #     ax.text(input_sinrs_db[-1]+0.5, y_pos, beamformer_name, fontsize=16, color=palette[i_beamformer])
# ax.set_xlim(input_sinrs_db[0], input_sinrs_db[-1]); ax.set_ylim(-20, 20)
# ax.set_xlabel("Input SINR"); ax.set_ylabel("Output SINR [dB]");
# ax.spines["top"].set_alpha(0.3)
# ax.spines["bottom"].set_alpha(0.3)
# ax.spines["right"].set_alpha(0.3)
# ax.spines["left"].set_alpha(.3)
# plt.legend()
# fig.savefig(filepath, dpi=600)

# filename = "ir_input_inr_{}dB".format(input_inr_db)
# filepath = os.path.join(folder, filename + ".jpg")
# fig = plt.figure(figsize=figsize); ax = fig.add_subplot(111);
# for i_beamformer, beamformer_name in enumerate(beamformer_list):
#     ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_ir_db"],
#                 label=beamformer_name, color=palette[i_beamformer])
#     y_pos = beamformers[beamformer_name]["average_ir_db"][-1] - 0.5 
#     ax.text(input_sinrs_db[-1]+0.5, y_pos, beamformer_name, fontsize=12, color=palette[i_beamformer])    
# ax.fill_between(input_sinrs_db, beamformers["DL-MPDR"]["average_ir_db"], 
#                 beamformers["MPDR"]["average_ir_db"], color=palette[0], alpha=0.2)    
# ax.set_xlim(input_sinrs_db[0], input_sinrs_db[-1]);
# ax.set_xlabel("Input SINR"); ax.set_ylabel("Output SINR [dB]");
# ax.spines["top"].set_alpha(0.3)
# ax.spines["bottom"].set_alpha(0.3)
# ax.spines["right"].set_alpha(0.3)
# ax.spines["left"].set_alpha(.3)
# fig.savefig(filepath, dpi=600)

# filename = "nr_input_inr_{}dB".format(input_inr_db)
# filepath = os.path.join(folder, filename + ".jpg")
# fig = plt.figure(figsize=figsize); ax = fig.add_subplot(111);
# for i_beamformer, beamformer_name in enumerate(beamformer_list):
#     ax.plot(input_sinrs_db, beamformers[beamformer_name]["average_nr_db"],
#                 label=beamformer_name, color=palette[i_beamformer])
#     y_pos = beamformers[beamformer_name]["average_nr_db"][-1] - 0.5
#     if beamformer_name == "DL-MVDR":
#         y_pos += 1
#     ax.text(input_sinrs_db[-1]+0.5, y_pos, beamformer_name, fontsize=12, color=palette[i_beamformer])    
# ax.fill_between(input_sinrs_db, beamformers["DL-MPDR"]["average_nr_db"], 
#                 beamformers["MPDR"]["average_nr_db"], color=palette[0], alpha=0.2)    
# ax.set_xlim(input_sinrs_db[0], input_sinrs_db[-1]);
# ax.set_xlabel("Input SINR"); ax.set_ylabel("Output SINR [dB]");
# ax.spines["top"].set_alpha(0.3)
# ax.spines["bottom"].set_alpha(0.3)
# ax.spines["right"].set_alpha(0.3)
# ax.spines["left"].set_alpha(.3)
# fig.savefig(filepath, dpi=600)

In [ ]:
# Visualize single frequency delay-sum
slide = "figures"
filename = "single_frequency_delaysum"
filepath = os.path.join(slide, filename + ".jpg")
frequency_bins = [127]
fig, ax = visualize_beampattern_1d(ds_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
                         signal_max_frequency, source_azimuths= azimuth_s)
fig.savefig(filepath, dpi=600)

In [ ]:
# Visualize time-frequency frame
slide = "figures"
filename = "received_tf_frame"
filepath = os.path.join(slide, filename + ".jpg")
visualize_tf(received_stft_multichannel[:, 0, :], sampling_frequency);
plt.savefig(filepath, dpi=600)

In [ ]:
visualize_tf(noise_stft_multichannel[:, 0, :], sampling_frequency);

In [ ]:
import soundfile as sf
def convert_tf_frames_to_signal(tf_frames, sampling_frequency, stft_params):
    stft_window = stft_params["window"]
    n_samples_per_frame = stft_params["n_samples_per_frame"]
    hop_size = stft_params["hop_size"]
    t, ss = istft(tf_frames, fs=sampling_frequency, 
        window=stft_window, nperseg=n_samples_per_frame, 
        noverlap=n_samples_per_frame-hop_size,
        nfft=n_samples_per_frame, boundary=True)
    return t, ss

def write_audio_file(signal, sampling_frequency, filename):
    signal = np.int16(signal/np.max(np.abs(signal)) * 32767)
    sf.write(filename, signal, sampling_frequency)
    
folder = "out"
filename = "received_demo"
filepath = os.path.join(folder, filename + ".wav")
t, received = convert_tf_frames_to_signal(received_stft_multichannel[:, 0, :], sampling_frequency, stft_params) 
write_audio_file(received, sampling_frequency, filepath)
filename = "source_demo"
filepath = os.path.join(folder, filename + ".wav")
t, received = convert_tf_frames_to_signal(source_stft_multichannel[:, 0, :], sampling_frequency, stft_params) 
write_audio_file(received, sampling_frequency, filepath)

In [ ]:
# Wideband delaysum results
slide = "figures"
filename = "wideband_delaysum_beampattern_1d"
filepath = os.path.join(slide, filename + ".jpg")
frequency_bins = [7, 31, 63, 127]
ax = visualize_beampattern_1d(ds_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, interference_azimuths=interference["azimuth"])
plt.savefig(filepath, dpi=600)

filename = "wideband_delaysum_beampattern_1d_average"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_1d_average(np.abs(ds_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, interference_azimuths=interference["azimuth"])
plt.savefig(filepath, dpi=600)

filename = "wideband_delaysum_beampattern_2d"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_2d(np.abs(ds_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);
plt.savefig(filepath, dpi=600)

folder = "out"
filename = "wideband_received_demo"
filepath = os.path.join(folder, filename + ".wav")
t, received = convert_tf_frames_to_signal(received_stft_multichannel[:, 0, :], sampling_frequency, stft_params) 
write_audio_file(received, sampling_frequency, filepath)

filename = "wideband_source_demo"
filepath = os.path.join(folder, filename + ".wav")
t, received = convert_tf_frames_to_signal(source_stft_multichannel[:, 0, :], sampling_frequency, stft_params) 
write_audio_file(received, sampling_frequency, filepath)

filename = "wideband_ds_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['DS']['out'][0][0], sampling_frequency, filepath)

In [ ]:
beamformers['DS']['out'][0][0]
Audio(beamformers['DS']['out'][0][0], rate=sampling_frequency, autoplay=True)

In [ ]:
mvdr_tf_beampattern = compute_tf_beampattern(beamformers["MVDR"]["weights"][0][0], scanning_steering_vectors)
mvdr_tf_beampattern_db = to_db(mvdr_tf_beampattern)

slide = "figures"
filename = "wideband_mvdr_beampattern_1d"
filepath = os.path.join(slide, filename + ".jpg")
frequency_bins = [7, 31, 63, 127]
ax = visualize_beampattern_1d(mvdr_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, interference_azimuths=interference["azimuth"])
plt.savefig(filepath, dpi=600)

filename = "wideband_mvdr_beampattern_1d_average"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_1d_average(np.abs(mvdr_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, interference_azimuths=interference["azimuth"])
plt.savefig(filepath, dpi=600)

filename = "wideband_mvdr_beampattern_2d"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_2d(np.abs(mvdr_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);
plt.savefig(filepath, dpi=600)

filename = "wideband_mvdr_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['MVDR']['out'][0][0], sampling_frequency, filepath)

In [ ]:
mpdr_tf_beampattern = compute_tf_beampattern(beamformers["MPDR"]["weights"][0][0], scanning_steering_vectors)
mpdr_tf_beampattern_db = to_db(mpdr_tf_beampattern)

slide = "figures"
filename = "wideband_mpdr_beampattern_1d"
filepath = os.path.join(slide, filename + ".jpg")
frequency_bins = [7, 31, 63, 127]
ax = visualize_beampattern_1d(mpdr_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, interference_azimuths=interference["azimuth"])
plt.savefig(filepath, dpi=600)

filename = "wideband_mpdr_beampattern_1d_average"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_1d_average(np.abs(mpdr_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, interference_azimuths=interference["azimuth"])
plt.savefig(filepath, dpi=600)

filename = "wideband_mpdr_beampattern_2d"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_2d(np.abs(mpdr_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);
plt.savefig(filepath, dpi=600)

folder = 'out'
filename = "wideband_mpdr_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['MPDR']['out'][0][0], sampling_frequency, filepath)

In [ ]:
dl_mvdr_tf_beampattern = compute_tf_beampattern(beamformers["DL-MVDR"]["weights"][0][0], scanning_steering_vectors)
dl_mvdr_tf_beampattern_db = to_db(dl_mvdr_tf_beampattern)

slide = "figures"
filename = "wideband_dl_mvdr_nc_beampattern_1d"
filepath = os.path.join(slide, filename + ".jpg")
frequency_bins = [7, 31, 63, 127]
ax = visualize_beampattern_1d(dl_mvdr_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, interference_azimuths=interference["azimuth"])
plt.savefig(filepath, dpi=600)

filename = "wideband_dl_mvdr_nc_beampattern_1d_average"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_1d_average(np.abs(dl_mvdr_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, interference_azimuths=interference["azimuth"])
plt.savefig(filepath, dpi=600)

filename = "wideband_dl_mvdr_nc_beampattern_2d"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_2d(np.abs(dl_mvdr_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);
plt.savefig(filepath, dpi=600)

folder = 'out'
filename = "wideband_dl_mvdr_nc_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['DL-MVDR']['out'][0][0], sampling_frequency, filepath)
beamformers['DL-MVDR']['sinr_db']

In [ ]:
dl_mpdr_tf_beampattern = compute_tf_beampattern(beamformers["DL-MPDR"]["weights"][0][0], scanning_steering_vectors)
dl_mpdr_tf_beampattern_db = to_db(dl_mpdr_tf_beampattern)

slide = "figures"
filename = "wideband_dl_mpdr_nc_beampattern_1d"
filepath = os.path.join(slide, filename + ".jpg")
frequency_bins = [7, 31, 63, 127]
ax = visualize_beampattern_1d(dl_mpdr_tf_beampattern_db[:, 0, :], scanning_azimuth_grid, frequency_bins, 
        signal_max_frequency, source_azimuths= azimuth_s, interference_azimuths=interference["azimuth"])
# plt.savefig(filepath, dpi=600)

filename = "wideband_dl_mpdr_nc_beampattern_1d_average"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_1d_average(np.abs(dl_mpdr_tf_beampattern[:, 0, :]), scanning_azimuth_grid, 
        frequency_range=(0, 63), source_azimuths=azimuth_s, interference_azimuths=interference["azimuth"])
# plt.savefig(filepath, dpi=600)

filename = "wideband_dl_mpdr_beampattern_2d"
filepath = os.path.join(slide, filename + ".jpg")
ax = visualize_beampattern_2d(np.abs(dl_mpdr_tf_beampattern[:, 0, :]), 
        scanning_azimuth_grid, signal_max_frequency);
# plt.savefig(filepath, dpi=600)

folder = 'out'
filename = "wideband_dl_mpdr_nc_input_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['DL-MPDR']['out'][0][0], sampling_frequency, filepath)
# Audio(beamformers['DL-MPDR']['out'][0][0], rate=sampling_frequency)
print(beamformers['DL-MPDR']['sinr_db'])

In [ ]:
filename = "wideband_ds_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['DS']['out'][0][0], sampling_frequency, filepath)

filename = "wideband_mvdr_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['MVDR']['out'][0][0], sampling_frequency, filepath)

folder = 'out'
filename = "wideband_mpdr_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['MPDR']['out'][0][0], sampling_frequency, filepath)

folder = 'out'
filename = "wideband_dl_mpdr_nc_input_inr_db_{}".format(input_inr_db)
filepath = os.path.join(folder, filename + ".wav")
write_audio_file(beamformers['DL-MPDR']['out'][0][0], sampling_frequency, filepath)